In [464]:
import numpy as np
import pandas as pd
import scipy as sp
import math
from sklearn.preprocessing import QuantileTransformer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

csv = None
csv = pd.read_csv("Train_and_Validate_EEG.csv", header=0)

print(len(csv))

# Drop the blank columns
csv = csv.dropna(axis=1, how='all')

# convert sex to binary encoding
csv["sex"] = csv['sex'].replace(['F','M'],[-1,1])

# Mean Imputation
csv["IQ"] = csv["IQ"].fillna(csv["IQ"].mean())
csv["education"] = csv["education"].fillna(csv["education"].mean())

# Make the X data
all_columns = csv.columns
x_columns = all_columns.drop("ID").drop("main.disorder").drop("specific.disorder").drop("eeg.date")
X = csv[x_columns]
#original_data = X[X.columns[:4]].reset_index(drop=True)
#combined_dataframe = pd.concat([original_data, X], axis=1, ignore_index=True)

# make the Y data
y = csv["main.disorder"]


# Split train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

df1 = pd.read_csv('final_dataset_train_all.csv')
df3 = pd.read_csv('final_dataset_test_all.csv')
df1.columns = [i.lower() for i in df1.columns]
df3.columns = [i.lower() for i in df3.columns]
df2 = X_train.copy().iloc[:,4:]
df2.dropna(axis=1, how='all', inplace=True)
df2.columns = [i.lower() for i in df2.columns]
df2['main.disorder'] = y_train #this column is just there to make matching up the data set easier, it will be dropped later
X_train_split_model = X_train.copy().iloc[:,0:4] #we'll train and stack this model on top later
X_test_split_model_eeg = X_test.copy().iloc[:,4:] #we're splitting up the data set and matching it up with the data from the paper
X_test_split_model_eeg['main.disorder'] = y_test #this column is just there to make matching up the data set easier, it will be dropped later
X_test_split_model_non_eeg = X_test.copy().iloc[:,0:4] #for testing the stacked model
X_val_split_model_eeg = X_val.copy().iloc[:,4:] #we're splitting up the data set and matching it up with the data from the paper
X_val_split_model_eeg['main.disorder'] = y_val #this column is just there to make matching up the data set easier, it will be dropped later
X_val_split_model_non_eeg = X_val.copy().iloc[:,0:4] #for testing the stacked model


852


<ipython-input-464-5371ecb04ba3>:19: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  csv["sex"] = csv['sex'].replace(['F','M'],[-1,1])


In [465]:
df2.columns = df1.columns
X_test_split_model_eeg.columns = df1.columns
X_val_split_model_eeg.columns = df1.columns
df = pd.concat([df1, df2, df3], axis=0)
df = df.drop_duplicates() #eliminate duplicate data after combining
y_train_df2 = y_train
y_train = df['main.disorder'].copy()
#df = df.copy().drop(columns=['main.disorder'])
#df2 = df2.copy().drop(columns=['main.disorder'])
#X_test_split_model_eeg = X_test_split_model_eeg.copy().drop(columns=['main.disorder'])
#X_val_split_model_eeg = X_val_split_model_eeg.copy().drop(columns=['main.disorder'])
means = []
stds = []
labels = []
mask = (df['main.disorder'] == 'Healthy control')
healthy_control_subset = df[mask]
healthy_control_subset = healthy_control_subset.copy().drop(columns=['main.disorder'])
df = df.copy().drop(columns=['main.disorder'])
df2 = df2.copy().drop(columns=['main.disorder'])
X_test_split_model_eeg = X_test_split_model_eeg.copy().drop(columns=['main.disorder'])
X_val_split_model_eeg = X_val_split_model_eeg.copy().drop(columns=['main.disorder'])
for column_name in df.columns:
  means.append(healthy_control_subset[column_name].mean())
  stds.append(healthy_control_subset[column_name].std())
  labels.append(column_name)

dfstats = pd.DataFrame([means,stds], columns=df.columns)
#X_train_split_model['age'] = X_train_split_model['age']**2
#X_train_split_model['education'] = np.log(X_train_split_model['education'])
#X_age = X_train_split_model['age']
_,fitted_lambda_age= sp.stats.yeojohnson(X_train_split_model['age'],lmbda=None)
X_train_split_model['age'] = X_train_split_model['age'].apply(lambda x: sp.stats.yeojohnson(x,fitted_lambda_age))
agemean = X_train_split_model['age'].mean()
agestd = X_train_split_model['age'].std()
_,fitted_lambda_education = sp.stats.yeojohnson(X_train_split_model['education'],lmbda=None)
X_train_split_model['education'] = X_train_split_model['education'].apply(lambda x: sp.stats.yeojohnson(x,fitted_lambda_education))
educationmean = X_train_split_model['education'].mean()
educationstd = X_train_split_model['education'].std()

In [466]:
for column_name in df.columns:
  df.loc[:, column_name] = (df[column_name] - dfstats[column_name][0]) / dfstats[column_name][1] #(dfstats[column_name][1] - dfstats[column_name][0])
  X_test_split_model_eeg.loc[:, column_name] = (X_test_split_model_eeg[column_name] - dfstats[column_name][0]) / dfstats[column_name][1] #(dfstats[column_name][1] - dfstats[column_name][0])
  X_val_split_model_eeg.loc[:, column_name] = (X_val_split_model_eeg[column_name] - dfstats[column_name][0]) / dfstats[column_name][1] #(dfstats[column_name][1] - dfstats[column_name][0])
  df2.loc[:, column_name] = (df2[column_name] - dfstats[column_name][0]) / dfstats[column_name][1] #(dfstats[column_name][1] - dfstats[column_name][0])

In [467]:
# modify y to take a number for each unique disorder
le = LabelEncoder()

# Fit the encoder to your data
le.fit(y_train)
y_train = le.transform(y_train)
y_train_df2 = le.transform(y_train_df2)
y_val = le.transform(y_val)
y_test = le.transform(y_test)

In [468]:
# XGBoost

import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
#from sklearn.linear_model import LogisticRegression
#from sklearn.multioutput import MultiOutputClassifier
#model = MultiOutputClassifier(LogisticRegression())

# Create XGBoost classifier
clf = xgb.XGBClassifier(
    objective='multi:softmax',
    n_estimators=10000, #this stops very early, maybe around 100 idk. I just have this there for now
    learning_rate=0.1,
    max_depth=100,
    tree_method="auto",
    early_stopping_rounds=10
)

# Train the model
clf.fit(df, y_train, eval_set=[(X_test_split_model_eeg, y_test),(X_val_split_model_eeg, y_val)])

# Make predictions
y_pred = clf.predict(X_val_split_model_eeg)

[0]	validation_0-mlogloss:1.90781	validation_1-mlogloss:1.90236
[1]	validation_0-mlogloss:1.88524	validation_1-mlogloss:1.88235
[2]	validation_0-mlogloss:1.87015	validation_1-mlogloss:1.85438
[3]	validation_0-mlogloss:1.84845	validation_1-mlogloss:1.84168
[4]	validation_0-mlogloss:1.83817	validation_1-mlogloss:1.82567
[5]	validation_0-mlogloss:1.82767	validation_1-mlogloss:1.81246
[6]	validation_0-mlogloss:1.81513	validation_1-mlogloss:1.79375
[7]	validation_0-mlogloss:1.80786	validation_1-mlogloss:1.78098
[8]	validation_0-mlogloss:1.80051	validation_1-mlogloss:1.76868
[9]	validation_0-mlogloss:1.79338	validation_1-mlogloss:1.75732
[10]	validation_0-mlogloss:1.78930	validation_1-mlogloss:1.75204
[11]	validation_0-mlogloss:1.78820	validation_1-mlogloss:1.74206
[12]	validation_0-mlogloss:1.78269	validation_1-mlogloss:1.73879
[13]	validation_0-mlogloss:1.77754	validation_1-mlogloss:1.73392
[14]	validation_0-mlogloss:1.77253	validation_1-mlogloss:1.72800
[15]	validation_0-mlogloss:1.76626	

In [469]:
# Calculate accuracy
from sklearn import metrics
#y_test = le.transform(y_test)

metrics.accuracy_score(y_pred, y_val)

0.39906103286384975

In [470]:
csv2 = pd.read_csv("Test_Set_EEG.csv", header=0)


In [ ]:
csv

,ID,sex,age,eeg.date,education,IQ,main.disorder,specific.disorder,AB.A.delta.a.FP1,AB.A.delta.b.FP2,...,COH.F.gamma.o.Pz.p.P4,COH.F.gamma.o.Pz.q.T6,COH.F.gamma.o.Pz.r.O1,COH.F.gamma.o.Pz.s.O2,COH.F.gamma.p.P4.q.T6,COH.F.gamma.p.P4.r.O1,COH.F.gamma.p.P4.s.O2,COH.F.gamma.q.T6.r.O1,COH.F.gamma.q.T6.s.O2,COH.F.gamma.r.O1.s.O2
0,1,F,31.55,2016.4.7,16.0,103.0,Mood disorder,Depressive disorder,48.843890,46.533704,...,84.825697,62.094883,49.639937,61.347225,81.630630,39.464753,72.168580,30.893891,74.038603,51.237679
1,2,M,25.00,2018.3.23,16.0,115.0,Healthy control,Healthy control,34.108015,22.838567,...,77.398455,48.282117,73.185133,66.742485,49.466312,59.270167,66.639938,40.297206,55.547526,63.630547
2,3,M,26.20,2015.4.10,16.0,83.0,Schizophrenia,Schizophrenia,31.084064,28.212342,...,91.510025,51.974239,62.571143,75.509166,56.460421,64.001966,78.424703,53.494830,66.581021,80.202968
3,4,M,21.00,2016.5.13,15.0,NaN,Obsessive compulsive disorder,Obsessive compulsitve disorder,14.310468,13.947459,...,79.855172,42.420095,56.473788,54.241183,69.561898,41.384408,66.332780,30.034691,65.659003,54.662177
4,5,F,28.15,2018.7.14,18.0,101.0,Healthy control,Healthy control,21.763093,23.938428,...,38.160508,21.319727,32.967416,33.401386,37.406879,27.080760,39.050046,14.826695,32.285940,41.015604
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
847,941,M,19.23,2017.9.15,12.0,79.0,Schizophrenia,Schizophrenia,22.432700,22.799740,...,70.625220,59.330855,56.657348,61.684146,81.194932,60.868446,76.268561,53.096066,82.856007,72.506823
848,942,M,37.14,2011.6.13,14.0,97.0,Trauma and stress related disorder,Posttraumatic stress disorder,12.887332,14.295926,...,80.159305,34.496443,45.412377,49.901553,62.663935,41.000232,69.587211,23.529430,72.031307,54.297598
849,943,M,20.08,2017.1.11,12.0,93.0,Mood disorder,Depressive disorder,20.055370,11.734445,...,84.826939,64.134452,75.336179,76.576526,74.856292,68.741795,79.043187,62.032961,79.282698,80.363086
850,944,F,33.50,2011.12.30,12.0,103.0,Mood disorder,Depressive disorder,26.430570,20.721589,...,58.693931,40.321880,42.252775,43.764353,49.365259,33.500596,44.200607,27.534399,40.861627,49.097135


In [471]:
csv2 = csv2.dropna(axis=1, how='all')

In [472]:
csv2_subset = csv2.copy().iloc[:,6:]

In [476]:
csv2_subset.columns = df2.columns

In [478]:
output = clf.predict(csv2_subset)

In [482]:
output_labels = le.inverse_transform(output)

In [483]:
output_pd = pd.DataFrame(output_labels)

In [484]:
output_pd.to_csv('output.csv', index=False)